In [ ]:
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import preprocessing
from scipy.interpolate import CubicSpline
import ast, sys
from scipy.stats import iqr
from sklearn.metrics import accuracy_score
import glob
from scipy import signal

In [ ]:
labels = pd.read_csv('challenge/task3/label.CSV', sep=';', decimal='.', header=0)

In [ ]:
labels

In [ ]:
def mergeOnDist(df1,df2):
    ret = np.zeros((len(df1),df1.shape[1]+df2.shape[1]-1))
    
    for i in range(len(df1)):
        dist = np.Inf
        for j in range(len(df2)):
            current_dist = np.abs(df1.iloc[i,0]-df2.iloc[j,0])
            if current_dist < dist:
                dist = current_dist
            else:
                break
        comodo = np.concatenate((df1.iloc[i,:].values,df2.iloc[j,1:]))
        ret[i] = comodo
    
    return pd.DataFrame(ret)

In [ ]:
def mergeOnKey(df1,df2):
    #df1 = df1.iloc[:,:-3]
    d1 = df1.rename(columns={0: 'merge_att'}).copy()
    d2 = df2.rename(columns={0: 'merge_att'}).copy()
    return pd.merge_asof(d1, d2, on='merge_att', direction="nearest")

In [ ]:
def readData(labels):
    data_path = 'challenge/task3/DANCE/'
    names = labels.Name.values
    comodo_df = []
    ret = []
    exercise = 0
    for number, file in enumerate(glob.glob(data_path+ '*txt')): 
        old_query = file.split('_')[1] + '_segm0' + str(int(file.split('-')[-1].split('_')[0]))
        break
    
    for number, file in enumerate(glob.glob(data_path+ '*txt')):            
        query = file.split('_')[1] + '_segm0' + str(int(file.split('-')[-1].split('_')[0]))
        sys.stdout.write("\r computed {%d/ %d} " % (number+1, 1191) )
        sys.stdout.flush()
        df = pd.read_csv(file, sep=' ', decimal='.',header=None,skiprows=3)
        
        if query == old_query: 
            comodo_df.append(df)
        else: 
            data = comodo_df[0].dropna(axis=0)
            for idx_cdf in range(len(comodo_df)-1):
                data = mergeOnKey(data, comodo_df[idx_cdf+1].dropna(axis=0))
            
            current_data = np.zeros((len(data),52)) 
            current_data[:,:data.shape[1]-1] = data.values[:,1:]
            for idxn, name in enumerate(names):
                if old_query in name:
                    lab = labels.Quality.iloc[idxn]
                    person = labels.Person.iloc[idxn]
                    break
                
            meta = np.array([lab, person,exercise]) 
            meta = np.tile(meta, (len(data), 1))
            all_data = np.concatenate((meta,current_data[:,1:]), axis=1)
            if len(ret) == 0:
                ret = all_data
            else:
                ret = np.concatenate((ret, all_data), axis=0)
            comodo_df = []
            comodo_df.append(df)
            old_query = query
            exercise += 1
                
    print()
    return ret

In [ ]:
%%time
X = readData(labels)

In [ ]:
X = pd.DataFrame(X)

In [ ]:
X

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,l,0,0,0.6640629768371582,-0.18505899608135226,0.0,0.0,0.0,152.86585998535156,-43.84146499633789,...,-1.0,1.0,1.0,3.0,2.0,8.0,30.0,83.0,14.0,5.0
1,l,0,0,0.5893549919128418,-0.31835898756980896,0.0,0.0,0.0,93.10975646972656,-31.646341323852536,...,-43.0,-5.0,4.0,4.0,-1.0,-12.0,-1.0,-17.0,-1.0,1.0
2,l,0,0,0.7172849774360657,-0.151855006814003,0.0,0.0,0.0,27.92683029174805,-24.207317352294922,...,-1.0,-1.0,-6.0,-14.0,-7.0,12.0,0.0,-12.0,-1.0,-3.0
3,l,0,0,0.5473629832267761,-0.1289059966802597,0.0,0.0,0.0,11.036585807800293,-19.207317352294922,...,-1.0,-1.0,-6.0,-14.0,-7.0,12.0,0.0,-12.0,-1.0,-3.0
4,l,0,0,0.6445310115814209,-0.16210900247097015,0.0,0.0,0.0,23.170732498168945,-10.670731544494627,...,6.0,-1.0,-1.0,-7.0,-1.0,6.0,5.0,-4.0,13.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76973,l,12,150,0.2993159890174866,-0.03222699835896492,0.0,0.0,0.0,-88.65853881835938,82.5,...,2.0,2.0,-1.0,0.0,-24.0,-9.0,-3.0,-6.0,-3.0,-1.0
76974,l,12,150,0.3198240101337433,-0.14111299812793732,0.0,0.0,0.0,-66.5243911743164,84.20732116699219,...,-1.0,0.0,-1.0,0.0,-24.0,-9.0,-3.0,-6.0,-3.0,-1.0
76975,l,12,150,0.4677729904651642,-0.171875,0.0,0.0,0.0,-63.84146499633789,89.69512176513672,...,0.0,-1.0,0.0,0.0,10.0,-5.0,1.0,-3.0,1.0,2.0
76976,l,12,150,0.5151370167732239,-0.11816400289535522,0.0,0.0,0.0,-92.19512176513673,92.13414764404298,...,0.0,-1.0,-4.0,-5.0,3.0,-36.0,-7.0,-2.0,1.0,-2.0


In [ ]:
codes, uniques = pd.factorize(X.iloc[:,0])
X.iloc[:,0] = codes
X

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,0,0,0,0.6640629768371582,-0.18505899608135226,0.0,0.0,0.0,152.86585998535156,-43.84146499633789,...,-1.0,1.0,1.0,3.0,2.0,8.0,30.0,83.0,14.0,5.0
1,0,0,0,0.5893549919128418,-0.31835898756980896,0.0,0.0,0.0,93.10975646972656,-31.646341323852536,...,-43.0,-5.0,4.0,4.0,-1.0,-12.0,-1.0,-17.0,-1.0,1.0
2,0,0,0,0.7172849774360657,-0.151855006814003,0.0,0.0,0.0,27.92683029174805,-24.207317352294922,...,-1.0,-1.0,-6.0,-14.0,-7.0,12.0,0.0,-12.0,-1.0,-3.0
3,0,0,0,0.5473629832267761,-0.1289059966802597,0.0,0.0,0.0,11.036585807800293,-19.207317352294922,...,-1.0,-1.0,-6.0,-14.0,-7.0,12.0,0.0,-12.0,-1.0,-3.0
4,0,0,0,0.6445310115814209,-0.16210900247097015,0.0,0.0,0.0,23.170732498168945,-10.670731544494627,...,6.0,-1.0,-1.0,-7.0,-1.0,6.0,5.0,-4.0,13.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76973,0,12,150,0.2993159890174866,-0.03222699835896492,0.0,0.0,0.0,-88.65853881835938,82.5,...,2.0,2.0,-1.0,0.0,-24.0,-9.0,-3.0,-6.0,-3.0,-1.0
76974,0,12,150,0.3198240101337433,-0.14111299812793732,0.0,0.0,0.0,-66.5243911743164,84.20732116699219,...,-1.0,0.0,-1.0,0.0,-24.0,-9.0,-3.0,-6.0,-3.0,-1.0
76975,0,12,150,0.4677729904651642,-0.171875,0.0,0.0,0.0,-63.84146499633789,89.69512176513672,...,0.0,-1.0,0.0,0.0,10.0,-5.0,1.0,-3.0,1.0,2.0
76976,0,12,150,0.5151370167732239,-0.11816400289535522,0.0,0.0,0.0,-92.19512176513673,92.13414764404298,...,0.0,-1.0,-4.0,-5.0,3.0,-36.0,-7.0,-2.0,1.0,-2.0


In [ ]:
X.iloc[:,0].value_counts()

0    38753
1    38225
Name: 0, dtype: int64

In [ ]:
X.rename(columns={ X.columns[0]: "label",X.columns[1]: "person",X.columns[2]: "exercise"},inplace=True)

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
selector = VarianceThreshold()
selector.fit_transform(X)

In [ ]:
X.shape

In [ ]:
X_new = X.copy()
X_new = X_new.loc[:,selector.get_support()].copy()

In [ ]:
X_new.shape

In [ ]:
X = X_new.copy()
X

In [ ]:
X.to_csv('challenge/raw_data.csv', sep=';', decimal='.', index=False, header=True)

In [ ]:
X = pd.read_csv('challenge/task3/raw_data.csv', sep=';', decimal='.', header=0)

In [ ]:
from scipy.interpolate import interp1d, CubicSpline

def interpolate(X,lenght):
    groups = X.groupby([X.label, X.person, X.exercise])
    if lenght == None:
        lenght = int(groups.size().mode())
    ret = []
    r, c = lenght, X.shape[1]
    for name, g in groups:
        comodo = np.zeros((r,c))
        comodo[:r,:3] = np.tile(g.iloc[0,:3].values, (r,1))

        x = g.index
        x_new = np.linspace(np.min(x),np.max(x), lenght)
        
        for i in range(3,c):
            y = g.iloc[:,i]
            f = CubicSpline(x,y)

            y_new = f(x_new)
            comodo[:,i] = y_new

        ret.append(comodo)

    ret = pd.DataFrame(np.vstack(ret), columns=X.columns)
    ret.sort_index(inplace=True)
    ret.reset_index(drop=True, inplace=True)
    return ret, lenght

In [ ]:
Xi, dim = interpolate(X, lenght=None)
dim

In [ ]:
Xi

In [ ]:
from statsmodels.graphics.tsaplots import acf
def extractFeatures(features):
    raw_features = features.shape[1]
    mean = features.mean().values
    var = features.var().values
    kurt = features.kurtosis().values
    skew = features.skew().values
    corr = features.corr().values[np.triu_indices(raw_features,k=1)]
    mad = features.mad().values
    sem = features.sem().values
    arr = features.values
    energy = np.sqrt(np.einsum('ij,ij->j',arr,arr))
    iqr_ = iqr(arr,axis=0)
    mi = features.min().values
    ma = features.max().values
    slopes = features.apply(lambda x: np.polyfit(features.index, x, 1)[0]).values

    frq_info = []
    for el in features[features.columns]:
        f_ = features[el]
        acf_ = np.nan_to_num(acf(f_,nlags=10)[1:])
        acf_mean = np.mean(acf_)
        acf_std = np.mean(acf_)
        
        fft = np.fft.fft(f_)
        amplitude_spectrum = np.abs(fft)
        phase_angle = np.angle(f_)
        

    frq_info.extend([acf_mean,acf_std,amplitude_spectrum[0].real,phase_angle[0].real,np.max(np.abs(fft[1:])),np.argmax(fft[1:])])
    frq_info.extend([pd.DataFrame(fft[1:]).abs().kurtosis().values[0],pd.DataFrame(fft[1:]).abs().skew().values[0], pd.DataFrame(fft[1:]).abs().mean(axis=0).values[0]])
  
    frq_info = np.asarray(frq_info)
    return np.concatenate((mean,var,kurt,skew,corr,mad,sem,energy,iqr_,mi,ma,slopes,frq_info)) 

def featureEngineering(X):
    groups = X.groupby([X.person, X.exercise])
    keys = list(groups.groups.keys())

    n_groups = groups.ngroups
    ret = np.zeros((n_groups,5403))

    for i in range(len(keys)):
        sys.stdout.write("\r computed {%d/ %d}" % (i+1, n_groups) )
        sys.stdout.flush()
        g = groups.get_group(list(keys)[i])

        features = g.iloc[:,3:]
        labels = g.iloc[0,:3].values

        f_per_launch = []
        dim_window = round(len(features)/2)
        step1 = round(dim_window/2)
        
        f_computed = extractFeatures(features) 
        f_per_exp = np.concatenate((labels,f_computed),axis=0)
           
        for j in range(0,dim_window,step1):
            f_computed = extractFeatures(features.iloc[j:j+dim_window,:])
            f_per_exp = np.concatenate((f_per_exp,f_computed),axis=0)
        
        step2 = round(step1/2)
        for j in range(0, step1,step2):
            f_computed = extractFeatures(features.iloc[j:j+dim_window,:])
            f_per_exp = np.concatenate((f_per_exp,f_computed),axis=0)
            
        f_per_exp = f_per_exp.reshape(1,-1) 
        ret[i,:] = f_per_exp
    print()
    return pd.DataFrame(ret)

In [ ]:
%%time
feat = featureEngineering(Xi) 

In [ ]:
feat.to_csv('challenge/features.csv', sep=';', decimal='.', index=False, header=True)

In [ ]:
feat = pd.read_csv('challenge/features.csv', sep=';', decimal='.', header=0)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
feat.head()

In [ ]:
def normByPerson(X):
    groups = X.groupby(['person'])
    ret = []
    for name, g in groups:
        s = g.iloc[:,3:]
        scaler = preprocessing.StandardScaler()
        new_values = g.values
        new_values[:,3:] = scaler.fit_transform(s.values)
        ret.append(new_values)

    ret = pd.DataFrame(np.asarray(np.vstack(ret)))
    ret.columns = X.columns  
    return ret

In [ ]:
feat.rename(columns={ feat.columns[0]: "label",feat.columns[1]: "person",feat.columns[2]: "exercise"},inplace=True)

In [ ]:
%%time
X_norm = normByPerson(feat.copy()) 

In [ ]:
feat.head()

In [ ]:
X_norm.head()

In [ ]:
X_norm.to_csv('challenge/X.csv', sep=';', decimal='.', index=False, header=True)

In [ ]:
X = pd.read_csv('challenge/X.csv', sep=';', decimal='.', header=0)

In [ ]:
def leave_1_out(X):
    grouped = X.groupby([X.person])

    for i, (name,group) in enumerate(grouped):
        test = group.copy()
        train = X.drop(test.index)

        mi = min(train.label.value_counts())
        train = train.groupby(train.label, group_keys=False).apply(lambda x: x.sample(min(len(x), mi)))

        yield train.index, test.index

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import SelectFromModel, VarianceThreshold
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score, balanced_accuracy_score
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTETomek


In [ ]:
cv = leave_1_out(X)
rf = RandomForestClassifier(random_state=42,n_estimators=1000)
pipeline = Pipeline(steps = [['var_tresh',VarianceThreshold()],
                                ['rf', rf]]
                       )

n_scores = cross_val_score(pipeline, X.iloc[:,3:], X.label, scoring='accuracy', cv=cv, n_jobs=32)
# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
import xgboost as xgb

In [ ]:
cv = leave_1_out(X)
model = xgb.XGBClassifier(use_label_encoder =False,objective = "binary:logistic",eval_metric = "logloss")
pipeline = Pipeline(steps = [['var_tresh',VarianceThreshold()],
                                ['model', model]]
                       )

n_scores = cross_val_score(pipeline, X.iloc[:,3:], X.label, scoring='accuracy', cv=cv, n_jobs=32)
# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
%%time
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTE, ADASYN
from scipy import stats
from imblearn.under_sampling import RandomUnderSampler

param_grid = {
    'model__n_estimators': [80,100,200,500,800,1000,1200,1500],
    'model__eta': [0.01, 0.03, 0.05, 0.1, 0.3]
}


model = xgb.XGBClassifier(random_state=42)

pipeline = Pipeline(steps = [['var_tresh',VarianceThreshold()],
                                ['model', model]]
                       )
#rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=2, random_state=36851234)
cv = leave_1_out(X)

clf = GridSearchCV(pipeline, param_grid, cv=cv, scoring=('balanced_accuracy'), n_jobs=4,verbose=1)
clf.fit(X.iloc[:,3:], X.label)

In [ ]:
cv = leave_1_out(X)
model = xgb.XGBClassifier(use_label_encoder =False,objective = "binary:logistic",eval_metric = "logloss",
                          n_estimators=80)
pipeline = Pipeline(steps = [['var_tresh',VarianceThreshold()],
                                ['model', model]]
                       )

n_scores = cross_val_score(pipeline, X.iloc[:,3:], X.label, scoring='accuracy', cv=cv, n_jobs=32)
# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

Accuracy: 0.722 (0.116)


In [ ]:
%%time
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTE, ADASYN
from scipy import stats
from imblearn.under_sampling import RandomUnderSampler

param_grid = {
    'model__max_depth': range(3,10,2),
}

model = xgb.XGBClassifier(random_state=42,n_estimators=80)

pipeline = Pipeline(steps = [['var_tresh',VarianceThreshold()],
                                ['model', model]]
                       )
cv = leave_1_out(X)

clf = GridSearchCV(pipeline, param_grid, cv=cv, scoring=('accuracy'), n_jobs=4,verbose=1)
clf.fit(X.iloc[:,3:], X.label)

In [ ]:
clf.best_params_

In [ ]:
cv = leave_1_out(X)
model = xgb.XGBClassifier(use_label_encoder =False,objective = "binary:logistic",eval_metric = "logloss",
                          n_estimators=80,max_depth=3,random_state=42)
pipeline = Pipeline(steps = [['var_tresh',VarianceThreshold()],
                                ['model', model]]
                       )

n_scores = cross_val_score(pipeline, X.iloc[:,3:], X.label, scoring='accuracy', cv=cv, n_jobs=32)
# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
%%time
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTE, ADASYN
from scipy import stats
from imblearn.under_sampling import RandomUnderSampler

param_grid = {
    'model__gamma': [0,0.1,0.2,0.3,0.4,3,5],
}


model = xgb.XGBClassifier(n_estimators=80,max_depth=3)

pipeline = Pipeline(steps = [['var_tresh',VarianceThreshold()],
                                ['model', model]]
                       )
#rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=2, random_state=36851234)
cv = leave_1_out(X)

clf = GridSearchCV(pipeline, param_grid, cv=cv, scoring=('accuracy'), n_jobs=4,verbose=1)
clf.fit(X.iloc[:,3:], X.label)

In [ ]:
clf.best_params_

In [ ]:
cv = leave_1_out(X)
model = xgb.XGBClassifier(use_label_encoder =False,objective = "binary:logistic",eval_metric = "logloss",
                          n_estimators=80,max_depth=3,gamma=0.1)
pipeline = Pipeline(steps = [['var_tresh',VarianceThreshold()],
                                ['model', model]]
                       )

n_scores = cross_val_score(pipeline, X.iloc[:,3:], X.label, scoring='accuracy', cv=cv, n_jobs=32)
# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
from sklearn.preprocessing import Normalizer

In [ ]:
cv = leave_1_out(X)
model = xgb.XGBClassifier(use_label_encoder =False,objective = "binary:logistic",eval_metric = "logloss")
pipeline = Pipeline(steps = [['var_tresh',VarianceThreshold()],
                             ['norm',Normalizer(norm='max')],
                                ['model', model]]
                       )

n_scores = cross_val_score(pipeline, X.iloc[:,3:], X.label, scoring='accuracy', cv=cv, n_jobs=32)
# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
def train(X, algorithm='xgboost'):
    if algorithm=='RF':
      n_features = X.iloc[:,3:].shape[1]
      sqrt1_3 = round(pow(n_features,1/3))
      sqrt3_4 = round(pow(n_features,2/3))
      params = ['auto',sqrt1_3,sqrt3_4]

      param_grid = {
          'model__max_features':params
          'model__min_samples_leaf':[1,3,5]
      }
      model = RandomForestClassifier(n_estimators=1000)

      pipeline = Pipeline(steps = [['var_tresh',VarianceThreshold()],
                                    ['model', model]]
                                   )

    elif algorithm=='xgboost':
      param_grid = {
      'model__eta': np.arange(0.01, 0.07, 0.01),
      'model__max_depth': [3,5,7,9],
      'model__gamma': [0,0.1,0.2],
      'model__subsample':[0.6,0.8,1],
      'model__colsample_bytree': [0.5,0.8,1]
      }

      model = xgb.XGBClassifier(use_label_encoder =False,objective = "binary:logistic",eval_metric = "logloss",n_estimators=1200) 
      pipeline = Pipeline(steps = [['var_tresh',VarianceThreshold()],
                                    ['norm',Normalizer(norm='max')],
                                    ['model', model]]
                                   )
    else:
      print('algorithm not valid')
    
    
    media_tot = np.zeros((10))
    media_f1 = np.zeros((10,2))
    media_mc = np.zeros((10))
    
    
    for mc in range(100):
        cv = leave_1_out(X)
        for i, (train_idx, test_idx) in enumerate(cv):
            X_train = X.iloc[train_idx]
            X_test = X.iloc[test_idx]
            
            X_train.reset_index(inplace=True,drop=True)
            cv_Val = leave_1_out(X_train)
            
            
            clf = GridSearchCV(pipeline, param_grid, cv=cv_Val, scoring=('accuracy'), n_jobs=32)
            clf.fit(X_train.iloc[:,3:], X_train.label)
            
            
            y_pred = clf.best_estimator_.predict(X_test.iloc[:,3:])
            current_acc = accuracy_score(X_test.label, y_pred)
            media_tot[mc] += current_acc
            
            f10, f11 = f1_score(X_test.label, y_pred,average=None)
            media_f1[mc,0] += f10
            media_f1[mc,1] += f11
            media_mc[mc] += matthews_corrcoef(X_test.label, y_pred)
                                   
            print(mc, i, current_acc)
            print(media_tot[mc]/(i+1), media_f1[mc,0]/(i+1), media_f1[mc,1]/(i+1),media_mc[mc]/(i+1))
            print()
   
    print(np.mean(media_tot), np.mean(media_f1,axis=0), np.mean(media_mc))

In [ ]:
X = pd.read_csv('challenge/X.csv', sep=';', decimal='.', header=0)
train(X)